# Dataclass

O decorador `@dataclass` do módulo `dataclasses` permite inicializar classes com atributos mais facilmente.  
Para isso, faz uso de **atributos de classe** com *type annotations*. O código simples abaixo:

In [1]:
from dataclasses import dataclass

@dataclass
class Pessoa:
    nome: str
    idade: int
    peso: float = 50

Equivale ao seguinte, com método `__init__` e `__repr__`:

In [2]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float = 50):
        self.nome = nome
        self.idade = idade
        self.peso = peso
    def __repr__(self):
        return f"Pessoa(nome={self.nome},idade={self.idade},peso={self.peso})" 

In [3]:
john = Pessoa("João",35,65)
john

Pessoa(nome=João,idade=35,peso=65)

# Usando `dataclass` junto com `property`

Normalmente é possível definir propriedades que são iniciadas com argumentos de mesmo nome.  
O código abaixo define a propriedade `nome` que é inicializada com o argumento `nome` do construtor:

In [4]:
class Pessoa:
    def __init__(self, nome: str):
        self.nome = nome
    
    @property
    def nome(self):
        return self._nome
    
    @nome.setter
    def nome(self, value):
        self._nome = value

Porém isso não é possível de ser feito com `dataclass`, uma vez que a propriedade é um atributo de classe.   
Isto é, a propriedade redefiniria o atributo de classe usado para definir a `dataclass` .    
Isso é problemático quando o atributo da `dataclass` possui um valor *default*

In [5]:
@dataclass
class Pessoa:
    nome: str = "Pedro"
    
    @property
    def nome(self):
        print('Recuperando o valor do nome, o qual é:')
        return self._nome
    
    @nome.setter
    def nome(self, value):
        print("Definindo valor de nome = ", value)
        self._nome = value

pessoa = Pessoa("João") # passando o atributo não há problema
print(pessoa.nome)

Definindo valor de nome =  João
Recuperando o valor do nome, o qual é:
João


In [6]:
pessoa = Pessoa()  # não passando o atributo, esperando recuperar o default
print(pessoa.nome) # aqui ve-se que perde-se a referência ao default

Definindo valor de nome =  <property object at 0x00000199B6DD49F0>
Recuperando o valor do nome, o qual é:


Existem duas opões para resolver esse problema.

## Opção 1: Definir as propriedades depois da `dataclass`

Propriedades são atributos de classe, logo pode-se defini-las depois da classe,   
usando o nome da classe e *getters* e *setters* com nomes padronizados.

In [7]:
@dataclass
class Pessoa:
    nome: str = "Pedro"
    
    def get_nome(self):
        print('Recuperando o valor do nome, o qual é:')
        return self._nome
    
    def set_nome(self, value):
        print("Definindo valor de nome = ", value)
        self._nome = value

# Usando o nome da classe para definir a propriedade,
# depois de definir a classe
Pessoa.nome = property(Pessoa.get_nome, Pessoa.set_nome)

pessoa = Pessoa() # aqui já se usa o valor default
print(pessoa.nome) # agora não há problema em recuperar a propriedade

Definindo valor de nome =  Pedro
Recuperando o valor do nome, o qual é:
Pedro


___
É possível definir um novo decorador que aplica a operação acima para todas as propriedades

In [8]:
from dataclasses import fields

def dataprops(cls):
    """A decorator to make dataclasses fields acting as properties
    getter and setter methods names must initate with `get_` and `set_`"""
    
    for field in fields(cls):
        setattr(cls,
                field.name,
                property(
                    getattr(cls,f'get_{field.name}'),
                    getattr(cls,f'set_{field.name}')
                    )
                )
    return cls

In [9]:
@dataprops
@dataclass
class Pessoa:
    nome: str = "Pedro"
    idade: int = 23
    
    def get_nome(self):
        print('Recuperando o valor do nome, o qual é:')
        return self._nome
    
    def set_nome(self, value):
        print("Definindo valor de nome = ", value)
        self._nome = value
    
    def get_idade(self):
        print('Recuperando o valor da idade, a qual é:')
        return self._idade
    
    def set_idade(self, value):
        print("Definindo valor de idade = ", value)
        self._idade = value


pessoa = Pessoa() # aqui já se usa o valor default
# agora não há problema em recuperar a propriedade
print('\n')
print(pessoa.nome)
print(pessoa.idade)


Definindo valor de nome =  Pedro
Definindo valor de idade =  23


Recuperando o valor do nome, o qual é:
Pedro
Recuperando o valor da idade, a qual é:
23


## Opção 2: Definir `dataclass` normalmente e herdando seus atributos

Essa opção usa duas classes com herança, mas faz uso do decorador `property` ,   
e ainda mantém os nomes das propriedades em seus métodos setter e getter

In [10]:
@dataclass
class Pessoa:
    nome: str = "Jonh Doe"
    idade: int = 35

class PessoaProperty(Pessoa):
    
    @property
    def nome(self):
        print('Recuperando o valor do nome, o qual é:')
        return self._nome
    
    @nome.setter
    def nome(self, value):
        print("Definindo valor de nome = ", value)
        self._nome = value
    
    @property
    def idade(self):
        print('Recuperando o valor da idade, a qual é:')
        return self._idade
    
    @idade.setter
    def idade(self, value):
        print("Definindo valor de idade = ", value)
        self._idade = value

pessoa = PessoaProperty()
print('\n')
print(pessoa.nome)
print(pessoa.idade)

Definindo valor de nome =  Jonh Doe
Definindo valor de idade =  35


Recuperando o valor do nome, o qual é:
Jonh Doe
Recuperando o valor da idade, a qual é:
35


Deve ser nesta ordem. Quando a ordem é trocada, perde-se a definição das propriedades

In [11]:
class PessoaProperty:
    
    @property
    def nome(self):
        print('Recuperando o valor do nome, o qual é:')
        return self._nome
    
    @nome.setter
    def nome(self, value):
        print("Definindo valor de nome = ", value)
        self._nome = value

@dataclass
class Pessoa(PessoaProperty):
    nome: str = "Jonh Doe"

pessoa = Pessoa()
pessoa.nome # não é mais propriedade

'Jonh Doe'